<a href="https://colab.research.google.com/github/Foruck/sentiment-analysis-demo/blob/attention/bert_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 准备工作

### 加载数据

上传
cn_train_data.h5
en_train_data.h5
cn_valid_data.h5
en_valid_data.h5
到自己的Google Drive


In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


### 配置文件目录环境

log：训练日志目录

bst_model：最优checkpoint



In [0]:
!apt-get install psmisc
!rm -rf log
!rm -rf bst_model
!mkdir log
!mkdir bst_model
!cp /content/gdrive/My\ Drive/cn_train_data.h5 cn_train_data.h5
!cp /content/gdrive/My\ Drive/en_train_data.h5 en_train_data.h5
!cp /content/gdrive/My\ Drive/cn_valid_data.h5 cn_valid_data.h5
!cp /content/gdrive/My\ Drive/en_valid_data.h5 en_valid_data.h5
!ls

Reading package lists... Done
Building dependency tree       
Reading state information... Done
psmisc is already the newest version (23.1-1).
0 upgraded, 0 newly installed, 0 to remove and 8 not upgraded.
bst_model	  cn_valid_data.h5  en_valid_data.h5  log
cn_train_data.h5  en_train_data.h5  gdrive	      sample_data


### 配置python运行环境

In [0]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install torch torchvision
import torch
print('Torch', torch.__version__, 'CUDA', torch.version.cuda)
print('Device:', torch.device('cuda:0'))

!pip install pytorch_pretrained_bert

  Using cached https://files.pythonhosted.org/packages/7e/60/66415660aa46b23b5e1b72bc762e816736ce8d7260213e22365af51e8f9c/torch-1.0.0-cp36-cp36m-manylinux1_x86_64.whl
tcmalloc: large alloc 1073750016 bytes == 0x60f88000 @  0x7f1fe8fde2a4 0x591a07 0x5b5d56 0x502e9a 0x506859 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x504c28 0x502540 0x502f3d 0x507641
    100% |████████████████████████████████| 61kB 3.0MB/s 
    100% |████████████████████████████████| 2.0MB 6.3MB/s 
  Found existing installation: Pillow 4.0.0
    Uninstalling Pillow-4.0.0:
      Successfully uninstalled Pillow-4.0.0
Torch 1.0.0 CUDA 9.0.176
Device: cuda:0
    100% |████████████████████████████████| 51kB 2.5MB/s 


## 开始

### 导入模块

In [0]:
import sys
import os
import datetime
import logging
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F
import torch.utils.data
import h5py
import numpy as np
import time
from torch.autograd import Variable
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM, BertAdam

### 模型超参数设置

embedding_length
Sentence_Max_Length
不可改动

In [0]:
LSTM_Hidden_Size = 256
embedding_length = 768
Sentence_Max_Length = 128

### 模型

In [0]:
class bertDataset(torch.utils.data.Dataset):
  def __init__(self, data_path):
    with h5py.File(data_path, 'r') as f:
      self.data = f['data'][:, :]
      self.mask = f['mask'][:, :]
      self.annot = f['annot'][:]
    
  def __len__(self):
    return self.annot.shape[0]
    
  def __getitem__(self, idx):
    return self.data[idx, :], self.mask[idx, :], self.annot[idx]

      
class myBert(torch.nn.Module):
  def __init__(self, embedding_length=768, bert_path='bert-base-chinese', window=[7], classes=2, use_cuda=True):
    super(myBert, self).__init__()
    self.use_cuda = use_cuda
    self.num_filter = len(window)

    # Bert model
    self.bert = BertModel.from_pretrained(bert_path)
    
    # Convolution layers
    conv1, conv2, conv3, conv4 = [], [], [], []
    for i in range(self.num_filter):
      conv1.append(nn.Conv2d(1, 256, (window[i], embedding_length), stride=(1, 1), padding=(int((window[i] - 1) / 2), 0))) # out n*64*128*1
      conv2.append(nn.Conv1d(256, 128, 3, stride=1, padding=1)) # out n*128*64
      conv3.append(nn.Conv1d(128, 64, 3, stride=1, padding=1)) # out n*64*32
      conv4.append(nn.Conv1d(64, 16, 1, stride=1, padding=0)) # out n*16*16
    self.conv1, self.conv2 = nn.ModuleList(conv1), nn.ModuleList(conv2)
    self.conv3, self.conv4 = nn.ModuleList(conv3), nn.ModuleList(conv4)
    for i in range(self.num_filter):
      init.kaiming_normal_(self.conv1[i].weight.data)
      init.kaiming_normal_(self.conv2[i].weight.data)
      init.kaiming_normal_(self.conv3[i].weight.data)
      init.kaiming_normal_(self.conv4[i].weight.data)

    self.dropout = nn.Dropout(p=0.5)

    # LSTM layers
    self.lstm1 = nn.LSTMCell(embedding_length, LSTM_Hidden_Size) # out n*1*LSTM_Hidden_Size
    self.lstm2 = nn.LSTMCell(embedding_length, LSTM_Hidden_Size)
    
    # Attention
    self.h1k1 = nn.Linear(LSTM_Hidden_Size, 64)
    self.k1e1 = nn.Linear(64, 1)
    self.h2k2 = nn.Linear(LSTM_Hidden_Size, 64)
    self.k2e2 = nn.Linear(64, 1)

    # FC layer
    self.fc = nn.Linear(LSTM_Hidden_Size * 2 + 256 * self.num_filter, classes)
    init.kaiming_normal_(self.fc.weight.data)
    self.fc.bias.data.fill_(0)
    
    self.simple_fc = nn.Linear(768, 2)
    
  def forward(self, inputs, mask):
    # Get Features
    inputs = self.bert(inputs, token_type_ids=None, attention_mask=mask, output_all_encoded_layers=False)[0]
    inputs = inputs.unsqueeze(1)
        
    # Go through Bi-LSTM
    n = inputs.shape[0]
    x0 = inputs.squeeze(1)
    if self.use_cuda:
      if not inputs.is_cuda:
        inputs = inputs.cuda()
        x0 = x0.cuda()
      cx1 = torch.zeros(n, LSTM_Hidden_Size).cuda()
      hx1 = torch.zeros(n, LSTM_Hidden_Size).cuda()
      hx2 = torch.zeros(n, LSTM_Hidden_Size).cuda()
      cx2 = torch.zeros(n, LSTM_Hidden_Size).cuda()
      hxs1 = torch.zeros((n, x0.shape[1], LSTM_Hidden_Size)).cuda()
      hxs2 = torch.zeros((n, x0.shape[1], LSTM_Hidden_Size)).cuda()
    else:
      if inputs.is_cuda:
        inputs = inputs.cpu()
        x0 = x0.cpu()
      cx1 = torch.zeros(n, LSTM_Hidden_Size)
      hx1 = torch.zeros(n, LSTM_Hidden_Size)
      cx2 = torch.zeros(n, LSTM_Hidden_Size)
      hx2 = torch.zeros(n, LSTM_Hidden_Size)
      hxs1 = torch.zeros((n, x0.shape[1], LSTM_Hidden_Size))
      hxs2 = torch.zeros((n, x0.shape[1], LSTM_Hidden_Size))
    
    for i in range(x0.shape[1]):
      hx1, cx1 = self.lstm1(x0[:, i, :], (hx1, cx1))
      hxs1[:, i, :] = hx1
      hx2, cx2 = self.lstm1(x0[:, x0.shape[1] - 1 - i, :], (hx2, cx2))
      hxs2[:, i, :] = hx2
      
    k1 = self.h1k1(hxs1) # n*128*64
    e1 = self.k1e1(k1).squeeze(2) # n*128*1 -> n*128
    e1 = F.softmax(e1, dim=1).unsqueeze(1) # n*1*128
    lstm1 = torch.matmul(e1, hxs1).squeeze(1)
    k2 = self.h2k2(hxs2) # n*128*64
    e2 = self.k2e2(k2).squeeze(2)
    e2 = F.softmax(e2, dim=1).unsqueeze(1)
    lstm2 = torch.matmul(e2, hxs2).squeeze(1)
    lstm_x = torch.cat((lstm1, lstm2), 1)
    
    # Go Through CNN
    x = []
    for i in range(self.num_filter):
      x.append(self.conv1[i](inputs))
      x[i] = x[i].squeeze(3)
      x[i] = F.relu(F.max_pool1d(x[i], kernel_size=2, stride=2))
      x[i] = self.conv2[i](x[i])
      x[i] = F.relu(F.max_pool1d(x[i], kernel_size=2, stride=2))
      x[i] = self.conv3[i](x[i])
      x[i] = F.relu(F.max_pool1d(x[i], kernel_size=2, stride=2))
      x[i] = self.conv4[i](x[i])
      x[i] = x[i].view(n, -1)
      x[i] = self.dropout(x[i])
    cnn_x = torch.cat(x, 1)
    
    x1 = torch.cat((cnn_x, lstm_x), 1)
    
    x1 = self.dropout(self.fc(x1))
    return x1

In [0]:
class enBert(torch.nn.Module):
  def __init__(self, embedding_length=768, bert_path='bert-base-uncased', window=[7], classes=2, use_cuda=True):
    super(enBert, self).__init__()
    self.use_cuda = use_cuda
    self.num_filter = len(window)

    # Bert model
    self.bert = BertModel.from_pretrained(bert_path)

    self.dropout = nn.Dropout(p=0.5)

    # LSTM layers
    self.lstm1 = nn.LSTMCell(embedding_length, LSTM_Hidden_Size) # out n*1*LSTM_Hidden_Size
    self.lstm2 = nn.LSTMCell(embedding_length, LSTM_Hidden_Size)
    
    # Attention
    self.h1k1 = nn.Linear(LSTM_Hidden_Size, 64)
    self.k1e1 = nn.Linear(64, 1)
    self.h2k2 = nn.Linear(LSTM_Hidden_Size, 64)
    self.k2e2 = nn.Linear(64, 1)

    # FC layer
    self.fc = nn.Linear(LSTM_Hidden_Size * 2, classes)
    init.kaiming_normal_(self.fc.weight.data)
    self.fc.bias.data.fill_(0)
    
    self.simple_fc = nn.Linear(768, 2)
    
  def forward(self, inputs, mask):
    # Get Features
    inputs = self.bert(inputs, token_type_ids=None, attention_mask=mask, output_all_encoded_layers=False)[0]
    inputs = inputs.unsqueeze(1)
        
    # Go through Bi-LSTM
    n = inputs.shape[0]
    x0 = inputs.squeeze(1)
    if self.use_cuda:
      if not inputs.is_cuda:
        inputs = inputs.cuda()
        x0 = x0.cuda()
      cx1 = torch.zeros(n, LSTM_Hidden_Size).cuda()
      hx1 = torch.zeros(n, LSTM_Hidden_Size).cuda()
      hx2 = torch.zeros(n, LSTM_Hidden_Size).cuda()
      cx2 = torch.zeros(n, LSTM_Hidden_Size).cuda()
      hxs1 = torch.zeros((n, x0.shape[1], LSTM_Hidden_Size)).cuda()
      hxs2 = torch.zeros((n, x0.shape[1], LSTM_Hidden_Size)).cuda()
    else:
      if inputs.is_cuda:
        inputs = inputs.cpu()
        x0 = x0.cpu()
      cx1 = torch.zeros(n, LSTM_Hidden_Size)
      hx1 = torch.zeros(n, LSTM_Hidden_Size)
      cx2 = torch.zeros(n, LSTM_Hidden_Size)
      hx2 = torch.zeros(n, LSTM_Hidden_Size)
      hxs1 = torch.zeros((n, x0.shape[1], LSTM_Hidden_Size))
      hxs2 = torch.zeros((n, x0.shape[1], LSTM_Hidden_Size))
    
    for i in range(x0.shape[1]):
      hx1, cx1 = self.lstm1(x0[:, i, :], (hx1, cx1))
      hxs1[:, i, :] = hx1
      hx2, cx2 = self.lstm1(x0[:, x0.shape[1] - 1 - i, :], (hx2, cx2))
      hxs2[:, i, :] = hx2
      
    k1 = self.h1k1(hxs1) # n*128*64
    e1 = self.k1e1(k1).squeeze(2) # n*128*1 -> n*128
    e1 = F.softmax(e1, dim=1).unsqueeze(1) # n*1*128
    lstm1 = torch.matmul(e1, hxs1).squeeze(1)
    k2 = self.h2k2(hxs2) # n*128*64
    e2 = self.k2e2(k2).squeeze(2)
    e2 = F.softmax(e2, dim=1).unsqueeze(1)
    lstm2 = torch.matmul(e2, hxs2).squeeze(1)
    lstm_x = torch.cat((lstm1, lstm2), 1)
   
    x1 = self.dropout(self.fc(lstm_x))
    return x1

### 训练与测试函数

In [0]:
def train(model, train_loader, optimizer, logger, epoch=0, print_every=100):
  model = model.train()
  loss_fn = nn.CrossEntropyLoss()
  all_loss, all_accuracy = 0.0, 0.0
  hit, cnt = 0, 0
  for i, (x, mask, target) in enumerate(train_loader):
    x = x.cuda().long()
    mask = mask.cuda().long()
    target = target.cuda().long()
    target = torch.clamp(target, min=0, max=1)
    
    scores = model(x, mask)
    loss = loss_fn(scores, target)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    pred = torch.argmax(scores, dim=1)
    hit = torch.sum(target == pred)
    accuracy = float(hit) / int(x.shape[0])
    if i % print_every == 0:
      logger.info('Epoch %d, Iter %d, loss=%.4f, acc=%.4f' % (epoch, i, loss, accuracy))
      print(time.strftime("%Y-%m-%d %H:%M:%S ", time.localtime()) + 'Epoch %d, Iter %d, loss=%.4f, acc=%.4f' % (epoch, i, loss, accuracy))
    
    all_loss += float(loss) * int(x.shape[0])
    all_accuracy += float(hit)
    cnt += int(x.shape[0])
    
  all_loss /= float(cnt)
  all_accuracy /= float(cnt)
  logger.info('Epoch %d, train_loss=%.4f, train_accuracy=%.4f' % (epoch, all_loss, all_accuracy))
  print(time.strftime("%Y-%m-%d %H:%M:%S ", time.localtime()) + 'Epoch %d, train_loss=%.4f, train_accuracy=%.4f' % (epoch, all_loss, all_accuracy))
  return model, optimizer

def evaluate(model, valid_loader, logger, epoch=0, print_every=100):
  model = model.eval()
  loss_fn = nn.CrossEntropyLoss()
  all_loss = 0.0
  hit, tot = 0, 0
  for i, (x, mask, target) in enumerate(valid_loader):
    x = x.cuda().long()
    mask = mask.cuda().long()
    target = target.cuda().long()
    target = torch.clamp(target, min=0, max=1)
    
    scores = model(x, mask)
    loss = loss_fn(scores, target)
    pred = torch.argmax(scores, dim=1)
    
    hit += float(torch.sum(target == pred))
    all_loss += float(loss) * int(x.shape[0])
    tot += int(x.shape[0])
  
  all_loss /= tot
  accuracy = float(hit) / tot
  logger.info('Epoch %d, valid_loss=%.4f, valid_accuracy=%.4f' % (epoch, all_loss, accuracy))
  print(time.strftime("%Y-%m-%d %H:%M:%S ", time.localtime()) + 'Epoch %d, valid_loss=%.4f, valid_accuracy=%.4f' % (epoch, all_loss, accuracy))
  return model, accuracy

### 查看显存使用状况

In [0]:
!nvidia-smi

Wed Jan  2 08:12:37 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 396.44                 Driver Version: 396.44                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   59C    P0    67W / 149W |   7848MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

### 训练参数设置

In [0]:
torch.set_default_tensor_type('torch.FloatTensor')
device = torch.device("cuda")
end_epoch = 60
tag = 'cn' # en英文，cn中文
lr = 1e-3 #可以调节
# checkpoint = 'gdrive/My Drive/cn_attention.pth'
# checkpoint = 'gdrive/My Drive/en_5e-05_2018-12-23_04-48-53.pth'
checkpoint = ''
fine_tune = 5e-5
bz = 30
warmup_proportion = 0.1
bst_acc = 0.0

### 数据集、模型加载

In [0]:
if tag == 'cn':
  train_set = bertDataset('cn_train_data.h5')
  train_loader = torch.utils.data.DataLoader(dataset=train_set, batch_size=bz, shuffle=True, num_workers=8)
  valid_set = bertDataset('cn_valid_data.h5')
  valid_loader = torch.utils.data.DataLoader(dataset=valid_set, batch_size=int(bz / 2), shuffle=True, num_workers=8)
  myModel = enBert(embedding_length, 'bert-base-chinese', use_cuda=True)
elif tag == 'en':
  train_set = bertDataset('en_train_data.h5')
  train_loader = torch.utils.data.DataLoader(dataset=train_set, batch_size=bz, shuffle=True, num_workers=8)
  valid_set = bertDataset('en_valid_data.h5')
  valid_loader = torch.utils.data.DataLoader(dataset=valid_set, batch_size=int(bz / 2), shuffle=True, num_workers=8)
  myModel = enBert(embedding_length, use_cuda=True)
  

myModel = myModel.cuda()
ignored_params = list(map(id, myModel.bert.parameters()))
base_params = filter(lambda p: id(p) not in ignored_params, myModel.parameters())
optimizer_grouped_parameters = [{'params': base_params, 'weight_decay': 0.01}, {'params': myModel.bert.parameters(), 'weight_decay': 0.0, 'lr': fine_tune}]
optimizer = BertAdam(optimizer_grouped_parameters, lr=lr, warmup=warmup_proportion, t_total=train_set.__len__())
  
if checkpoint != '':
  state = torch.load(checkpoint)
  myModel.load_state_dict(state['model_state'])
  optimizer.load_state_dict(state['optim_state'])
  epoch = state['epoch']
  bst_acc = state['acc']
else:
  epoch = 0

### log、checkpoint存储目录设置

In [0]:
job_name =  '_'.join([tag, str(lr), datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')])
save_path = 'bst_model/' + job_name + '.pth'
log_path = 'log/' + job_name + '.log'
logger = logging.getLogger(__name__)
logger.setLevel(level = logging.INFO)
handler = logging.FileHandler(log_path)
handler.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)
logger.info("Start print log")
print(log_path)
print(epoch)
print(bst_acc)

log/cn_0.001_2019-01-02_08-12-47.log
0
0.0


### 开始训练

In [0]:
for i in range(epoch, end_epoch):
  logger.info('=> Epoch %d, lr = %0.6f <=' % (i, lr))
  myModel, optimizer = train(myModel, train_loader, optimizer, logger, epoch=i, print_every=50)
  myModel, accuracy = evaluate(myModel, valid_loader, logger, epoch=i)
  # break
  
  if (accuracy > bst_acc):
    bst_acc = accuracy
    state = {'model_state': myModel.state_dict(), 'epoch': i, 'optim_state': optimizer.state_dict(), 'acc': bst_acc}
    torch.save(state, save_path)
  
logger.info("Finish")


2019-01-02 08:12:50 Epoch 0, Iter 0, loss=0.8658, acc=0.4667
2019-01-02 08:14:25 Epoch 0, Iter 50, loss=0.6304, acc=0.6667
2019-01-02 08:16:00 Epoch 0, Iter 100, loss=0.4329, acc=0.7667
2019-01-02 08:17:35 Epoch 0, Iter 150, loss=0.5266, acc=0.7667
2019-01-02 08:19:11 Epoch 0, Iter 200, loss=0.4583, acc=0.8667
2019-01-02 08:20:46 Epoch 0, Iter 250, loss=0.4582, acc=0.7333
2019-01-02 08:22:21 Epoch 0, Iter 300, loss=0.2874, acc=0.8000
2019-01-02 08:23:56 Epoch 0, Iter 350, loss=0.2923, acc=0.8667
2019-01-02 08:25:31 Epoch 0, Iter 400, loss=0.6600, acc=0.5333
2019-01-02 08:27:07 Epoch 0, Iter 450, loss=0.4789, acc=0.7333
2019-01-02 08:28:42 Epoch 0, Iter 500, loss=0.6297, acc=0.7667
2019-01-02 08:30:17 Epoch 0, Iter 550, loss=0.4269, acc=0.8333
2019-01-02 08:31:57 Epoch 0, Iter 600, loss=0.3250, acc=0.8000
2019-01-02 08:33:33 Epoch 0, Iter 650, loss=0.4460, acc=0.6667
2019-01-02 08:35:08 Epoch 0, Iter 700, loss=0.5740, acc=0.6667
2019-01-02 08:36:43 Epoch 0, Iter 750, loss=0.3808, acc=0.


 ### 自行添加：/bst_model、/log内文件保存到google drive或本地




In [0]:
# TO DO
# !cp bst_model/en_5e-05_2018-12-23_04-48-53.pth gdrive/My\ Drive/en_5e-05_2018-12-23_04-48-53.pth
# cp bst_model/cn_0.001_2019-01-02_08-12-47.pth gdrive/My\ Drive/cn_attention.pth